# This file runs on ***Google Colab***
## Before running this file, the Data Source file needs to be placed at the same level as this file

### Data file name: PySpark_DataFile/hotel_reviews_for_NLP4_Star_Rating.zip/hotel_reviews_for_NLP4_Star_Rating.csv

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downl

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySpark_NLP_Attempt2a").getOrCreate()

In [3]:
# Read in data 
from pyspark import SparkFiles
file_path = "hotel_reviews_for_NLP4_Star_Rating.csv"
spark.sparkContext.addFile(file_path)
df = spark.read.csv(SparkFiles.get("hotel_reviews_for_NLP4_Star_Rating.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------------+-----------+
|              Review|Star_Rating|
+--------------------+-----------+
| I am so angry th...|          2|
| No real complain...|          4|
| Rooms are nice b...|          4|
| My room was dirt...|          2|
| You When I booke...|          4|
| Backyard of the ...|          4|
| Cleaner did not ...|          3|
| Apart from the p...|          5|
| Even though the ...|          3|
| The aircondition...|          4|
| Nothing all grea...|          5|
| 6 30 AM started ...|          3|
| The floor in my ...|          3|
| This hotel is be...|          5|
| The staff in the...|          5|
| This hotel is aw...|          5|
| Very steep steps...|          3|
| We did not like ...|          4|
| Public areas are...|          4|
| We had issues wi...|          4|
+--------------------+-----------+
only showing top 20 rows



In [4]:
# Import functions
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

In [5]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature
data_df = df.withColumn('length', length(df['Review']))
data_df.show()

+--------------------+-----------+------+
|              Review|Star_Rating|length|
+--------------------+-----------+------+
| I am so angry th...|          2|  1913|
| No real complain...|          4|   611|
| Rooms are nice b...|          4|   301|
| My room was dirt...|          2|  1221|
| You When I booke...|          4|   774|
| Backyard of the ...|          4|   186|
| Cleaner did not ...|          3|   235|
| Apart from the p...|          5|   157|
| Even though the ...|          3|   162|
| The aircondition...|          4|   312|
| Nothing all grea...|          5|   568|
| 6 30 AM started ...|          3|   430|
| The floor in my ...|          3|   152|
| This hotel is be...|          5|   329|
| The staff in the...|          5|   229|
| This hotel is aw...|          5|   413|
| Very steep steps...|          3|   270|
| We did not like ...|          4|   623|
| Public areas are...|          4|   166|
| We had issues wi...|          4|   620|
+--------------------+-----------+

In [6]:
# Create all the features to the data set
convert_review_output_to_numeric_labels = StringIndexer(inputCol='Star_Rating',outputCol='label')
tokenizer = Tokenizer(inputCol="Review", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [7]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [8]:
# Create and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[convert_review_output_to_numeric_labels, tokenizer, stopremove, hashingTF, idf, clean_up])

In [9]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)

# Show "Combined_Review" and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  3.0|(262145,[2437,302...|
|  1.0|(262145,[4714,514...|
|  1.0|(262145,[22346,23...|
|  3.0|(262145,[1797,230...|
|  1.0|(262145,[14870,20...|
|  1.0|(262145,[9781,304...|
|  2.0|(262145,[21641,34...|
|  0.0|(262145,[25789,43...|
|  2.0|(262145,[22815,31...|
|  1.0|(262145,[2437,216...|
|  0.0|(262145,[9129,181...|
|  2.0|(262145,[1696,383...|
|  2.0|(262145,[1729,216...|
|  0.0|(262145,[15370,23...|
|  0.0|(262145,[6957,304...|
|  0.0|(262145,[5765,218...|
|  2.0|(262145,[3280,110...|
|  1.0|(262145,[329,9129...|
|  1.0|(262145,[11941,17...|
|  1.0|(262145,[17435,21...|
+-----+--------------------+
only showing top 20 rows



In [10]:
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3], 21)

In [11]:
from pyspark.ml.classification import NaiveBayes
# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [12]:
# Transform the data with the testing data
test_results = predictor.transform(testing)
test_results.show(10)

+------+-----------+------+-----+----------+-----------+--------------+--------------+--------------------+--------------------+--------------------+----------+
|Review|Star_Rating|length|label|token_text|stop_tokens|    hash_token|     idf_token|            features|       rawPrediction|         probability|prediction|
+------+-----------+------+-----+----------+-----------+--------------+--------------+--------------------+--------------------+--------------------+----------+
|      |          4|     1|  1.0|        []|         []|(262144,[],[])|(262144,[],[])|(262145,[262144],...|[-0.9122641906855...|[0.57719519675686...|       0.0|
|      |          5|     1|  0.0|        []|         []|(262144,[],[])|(262144,[],[])|(262145,[262144],...|[-0.9122641906855...|[0.57719519675686...|       0.0|
|      |          5|     1|  0.0|        []|         []|(262144,[],[])|(262144,[],[])|(262145,[262144],...|[-0.9122641906855...|[0.57719519675686...|       0.0|
|      |          3|     2|  2.0| 

In [13]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" %acc)

Accuracy of model at predicting reviews was: 0.612702
